In [134]:
# compute equivalence classes based on a gff annotation
import os

In [158]:
def evaluate_interval(iv,oivs): # takes current interval and a sorted list of original intervals
    res = []
    for o in oivs:
        if iv[1]>o[0] and o[1]>iv[0]:
            res.append(o[2])
    return res

# TODO: need to use oivs to add information about which transcripts are shared by an equivalence class
def process_locus(eps,oivs):
    interval = []
    res = []
    for i,e in enumerate(eps): # iterate over and collect intervals
        if len(interval)==2: # full interval found, can be evaluated and reset
            segs = evaluate_interval(interval,oivs)
            if not len(segs)==0:
                res.append(interval+segs)
                tmp=interval[1]
                if not e[0]==0:
                    interval=[tmp,i]
                else:
                    interval=[tmp]
            else:
                tmp=interval[1]
                if not e[0]==0:
                    interval=[tmp,i]
                else:
                    interval=[tmp]
        elif not e[0]==0:
            interval.append(i)
        else:
            continue
    if len(interval)==2:
        segs = evaluate_interval(interval,oivs)
        if not len(segs)==0:
            res.append(interval+segs)
        
    return res

def report(segs,outFP,geneID,chrID,strand):
    if len(segs)>0:
        for s in segs:
#             print(s)
            outFP.write("\t".join([chrID,strand,str(s[0]),str(s[1]),str(len(s[2])),geneID,";".join(list(s[2]))])+"\n")

In [159]:
endpoints=[]
original_intervals = set() # these contain the transcript boundaries (without separation into exons and introns). 
# This is necessary in order to preserve splicejunction information whenver possible
chrom = ""
strand = "+"
ls = 0 # locus start
le = 0 # locus end
geneID = ""
ts = 0
te = 0
transID = ""
transPar= ""

inputFP = "/media/data/genomicData/hg38/annotations/chess2.2.gff"
outFP = "./chess2.2.ec"

ecFP = open(outFP,"w+")

with open(inputFP,"r") as gff:
    for line in gff.readlines():
        if(line[0]=="#"):
            continue
        else:
            lineCols = line.split("\t")
            if lineCols[2] == "gene":
                final = process_locus(endpoints,original_intervals) # process the intervals computed for the previous locus
                final = [[x[0]+ls,x[1]+ls,tuple(x[2:])] for x in final]
                report(final,ecFP,geneID,chrom,strand)
                # get some basic information about the locus and initialize things
                chrom = lineCols[0]
                strand = lineCols[6]
                ls = int(lineCols[3])
                le = int(lineCols[4])
                geneID = lineCols[8].strip().split("ID=")[1].split(";")[0]
#                 print(geneID)
                
                endpoints=[[0,set()] for x in range(ls,le+1,1)]
                original_intervals=set()
                
            elif lineCols[2] == "transcript":
                transID = lineCols[8].strip().split("ID=")[1].split(";")[0]
                transPar = lineCols[8].strip().split("Parent=")[1].split(";")[0]
                ts = int(lineCols[3])
                te = int(lineCols[4])
                
                # add to original intervals
#                 original_intervals.add((ts-1,te-1,))
                
            elif lineCols[2] == "exon":
                parent = lineCols[8].strip().split("Parent=")[1].split(";")[0]
                start = int(lineCols[3])
                end = int(lineCols[4])
                endpoints[start-ls][0]+=1
                endpoints[start-ls][1].add(parent)
                endpoints[end-ls][0]+=1
                endpoints[end-ls][1].add(parent)
                
                # add to original intervals
                original_intervals.add((start-ls,end-ls,parent))
            else:
                continue
                
    final = process_locus(endpoints,original_intervals)
    final = [[x[0]+ls,x[1]+ls,tuple(x[2:])] for x in final]
    report(final,outFP,geneID,chrom,strand)

IndexError: list index out of range

In [137]:
final

[[10474946, 10474947, ('CHS.360.2', 'CHS.360.1')],
 [10474947, 10474969, ('CHS.360.2', 'CHS.360.3', 'CHS.360.1')],
 [10474969, 10474982, ('CHS.360.2', 'CHS.360.3', 'CHS.360.1', 'CHS.360.4')],
 [10474982,
  10475002,
  ('CHS.360.2', 'CHS.360.3', 'CHS.360.5', 'CHS.360.1', 'CHS.360.4')],
 [10475002, 10475080, ('CHS.360.4',)],
 [10475202, 10475740, ('CHS.360.6',)],
 [10484499, 10484965, ('CHS.360.7',)],
 [10484965, 10485115, ('CHS.360.8', 'CHS.360.7')],
 [10494874, 10495116, ('CHS.360.9',)],
 [10495274,
  10495321,
  ('CHS.360.8',
   'CHS.360.2',
   'CHS.360.3',
   'CHS.360.7',
   'CHS.360.5',
   'CHS.360.6',
   'CHS.360.4',
   'CHS.360.9',
   'CHS.360.1')],
 [10521230, 10521231, ('CHS.360.12',)],
 [10521231, 10521607, ('CHS.360.12', 'CHS.360.13')],
 [10521607, 10524300, ('CHS.360.13',)],
 [10524406, 10524483, ('CHS.360.12', 'CHS.360.13')],
 [10528251, 10528359, ('CHS.360.13',)],
 [10536005, 10536060, ('CHS.360.14',)],
 [10536213,
  10536297,
  ('CHS.360.1',
   'CHS.360.2',
   'CHS.360.13'